# OCLC Parse MARC add ISBN




In [9]:
import pandas as pd
import pymarc
import io


In [10]:
path_to_tsv = "/Users/m/Downloads/data-tmp/iowa_writers_workshop-hathitrust_metadata.tsv"
isbn = 'isbns'

In [11]:
def add_auth_name(d):

    # add checks here
    if pd.isnull(d[isbn]) == False:
        return d

    # do we have some marc data?
    if type(d['oclc_marc']) == str:   
        # the pymarc library expects a file to open, we don't have files we have strings, 
        # so make a file like object and put our string into it so we can parse it
        with io.StringIO() as f:
            f.write(d['oclc_marc'])
            f.seek(0)
            # parse it, its returns a list of records, but we only have one, so take the 0 index
            record = pymarc.marcxml.parse_xml_to_array(f)[0]

            field = None
            if '020' in record:
                if 'a' in record['020']:
                    
                    i = record['020']['a']
                    i = i.split("(")[0].strip()
                    d[isbn] = i

    else:
        #print("No MARC data to parse:",d)
        pass

    return d

In [12]:
# load the tsv
df = pd.read_csv(path_to_tsv, sep='\t', header=0, low_memory=False)
df.drop(df.filter(regex="Unname"),axis=1, inplace=True)

# run our function over all of them
df = df.apply(lambda d: add_auth_name(d),axis=1 )  
# # overwrite back out
df.to_csv(path_to_tsv, sep='\t')

